# Introduction to Python  

## Accessing Relational Databases

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd

import sqlite3

### Acessing [SQLite](https://docs.python.org/3/library/sqlite3.html)  

In [2]:
conn = sqlite3.connect(os.path.join("..","Data",'sqlite_example.db'))

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[]


In [5]:
# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [8]:
conn = sqlite3.connect(os.path.join("..","Data",'sqlite_example.db'))
cur = conn.cursor()

In [9]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('stocks',)]


In [10]:
t = ('RHAT',) #tuple with just one element
cur.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(cur.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [11]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
cur.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

In [12]:
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


#### Using Pandas

In [13]:
conn = sqlite3.connect(os.path.join("..","Data",'sqlite_example.db'))
df = pd.read_sql_query("SELECT * from stocks ORDER BY price", conn)

# verify that result of SQL query is stored in the dataframe
print(df.head())

conn.close()

         date trans symbol    qty  price
0  2006-01-05   BUY   RHAT  100.0  35.14


Deleting the database file:  

In [15]:
os.remove(os.path.join("..","Data",'sqlite_example.db'))

### Accessing existing [Database](https://www.sqlitetutorial.net/sqlite-sample-database/):

![Chinook Schema](../Data/sqlite-sample-database-color.jpg)

In [17]:
conn = sqlite3.connect(os.path.join("..","Data",'chinook.sqlite'))
cur = conn.cursor()

In [18]:
query = '''
SELECT name
FROM sqlite_master 
WHERE type ='table' AND name NOT LIKE 'sqlite_%';
'''

cur.execute(query)
for c in cur.fetchall():
    print(c)

('albums',)
('artists',)
('customers',)
('employees',)
('genres',)
('invoices',)
('invoice_items',)
('media_types',)
('playlists',)
('playlist_track',)
('tracks',)


In [19]:
query = '''
SELECT sql 
FROM sqlite_master 
WHERE name = 'albums';
'''

cur.execute(query)
for c in cur.fetchall():
    print(c[0])

CREATE TABLE "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [20]:
query = '''
SELECT albumid, title
FROM albums
ORDER BY title
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

(156, '...And Justice For All')
(257, '20th Century Masters - The Millennium Collection: The Best of Scorpions')
(296, 'A Copland Celebration, Vol. I')
(94, 'A Matter of Life and Death')
(95, 'A Real Dead One')
(96, 'A Real Live One')
(285, 'A Soprano Inspired')
(139, 'A TempestadeTempestade Ou O Livro Dos Dias')
(203, 'A-Sides')
(160, 'Ace Of Spades')


In [21]:
query = '''
SELECT trackid, name, composer, unitprice
FROM tracks
LIMIT 10 OFFSET 10;
'''

cur.execute(query)
for c in cur.fetchall():
    print(c)

(11, 'C.O.D.', 'Angus Young, Malcolm Young, Brian Johnson', 0.99)
(12, 'Breaking The Rules', 'Angus Young, Malcolm Young, Brian Johnson', 0.99)
(13, 'Night Of The Long Knives', 'Angus Young, Malcolm Young, Brian Johnson', 0.99)
(14, 'Spellbound', 'Angus Young, Malcolm Young, Brian Johnson', 0.99)
(15, 'Go Down', 'AC/DC', 0.99)
(16, 'Dog Eat Dog', 'AC/DC', 0.99)
(17, 'Let There Be Rock', 'AC/DC', 0.99)
(18, 'Bad Boy Boogie', 'AC/DC', 0.99)
(19, 'Problem Child', 'AC/DC', 0.99)
(20, 'Overdose', 'AC/DC', 0.99)


In [22]:
query = '''
SELECT name, milliseconds, albumid
FROM tracks
ORDER BY milliseconds DESC, albumid ASC;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('Occupation / Precipice', 5286953, 227)
('Through a Looking Glass', 5088838, 229)
('Greetings from Earth, Pt. 1', 2960293, 253)
('The Man With Nine Lives', 2956998, 253)
('Battlestar Galactica, Pt. 2', 2956081, 253)
('Battlestar Galactica, Pt. 1', 2952702, 253)
('Murder On the Rising Star', 2935894, 253)
('Battlestar Galactica, Pt. 3', 2927802, 253)
('Take the Celestra', 2927677, 253)
('Fire In Space', 2926593, 253)


In [23]:
query = '''
SELECT  DISTINCT city, country
FROM customers
ORDER BY country;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('Buenos Aires', 'Argentina')
('Sidney', 'Australia')
('Vienne', 'Austria')
('Brussels', 'Belgium')
('São José dos Campos', 'Brazil')
('São Paulo', 'Brazil')
('Rio de Janeiro', 'Brazil')
('Brasília', 'Brazil')
('Montréal', 'Canada')
('Edmonton', 'Canada')


In [24]:
query = '''
SELECT name, milliseconds, bytes, albumid
FROM tracks
WHERE albumid = 1
AND milliseconds > 250000;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('For Those About To Rock (We Salute You)', 343719, 11170334, 1)
('Evil Walks', 263497, 8611245, 1)
('Breaking The Rules', 263288, 8596840, 1)
('Spellbound', 270863, 8817038, 1)


In [25]:
query = '''
SELECT name, albumid, composer
FROM tracks
WHERE composer LIKE '%Smith%'
ORDER BY albumid;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('Restless and Wild', 3, 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman')
('Princess of the Dawn', 3, 'Deaffy & R.A. Smith-Diesel')
('Killing Floor', 19, 'Adrian Smith')
('Machine Men', 19, 'Adrian Smith')
('2 Minutes To Midnight', 95, 'Adrian Smith/Bruce Dickinson')
('Can I Play With Madness', 96, 'Adrian Smith/Bruce Dickinson/Steve Harris')
('The Evil That Men Do', 96, 'Adrian Smith/Bruce Dickinson/Steve Harris')
('The Wicker Man', 97, 'Adrian Smith/Bruce Dickinson/Steve Harris')
('The Fallen Angel', 97, 'Adrian Smith/Steve Harris')
('Wildest Dreams', 98, 'Adrian Smith/Steve Harris')


In [26]:
query = '''
SELECT name, albumid, mediatypeid
FROM tracks
WHERE mediatypeid IN (2, 3);
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('Balls to the Wall', 2, 2)
('Fast As a Shark', 3, 2)
('Restless and Wild', 3, 2)
('Princess of the Dawn', 3, 2)
('Welcome to the Jungle', 90, 2)
("It's So Easy", 90, 2)
('Nightrain', 90, 2)
('Out Ta Get Me', 90, 2)
('Mr. Brownstone', 90, 2)
('Paradise City', 90, 2)


In [27]:
query = '''
SELECT InvoiceId, BillingAddress, Total
FROM invoices
WHERE Total BETWEEN 14.91 and 18.86    
ORDER BY Total; 
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

(193, 'Berger Straße 10', 14.91)
(103, '162 E Superior Street', 15.86)
(208, 'Ullevålsveien 14', 15.86)
(306, 'Klanova 9/506', 16.86)
(313, '68, Rue Jouvence', 16.86)
(88, 'Calle Lira, 198', 17.91)
(89, 'Rotenturmstraße 4, 1010 Innere Stadt', 18.86)
(201, '319 N. Frances Street', 18.86)


In [28]:
query = '''
SELECT InvoiceId, BillingAddress, Total
FROM invoices
WHERE Total NOT BETWEEN 1 and 20
ORDER BY Total;  
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

(6, 'Berger Straße 10', 0.99)
(13, '1600 Amphitheatre Parkway', 0.99)
(20, '110 Raeburn Pl', 0.99)
(27, '5112 48 Street', 0.99)
(34, 'Praça Pio X, 119', 0.99)
(41, 'C/ San Bernardo 85', 0.99)
(48, '796 Dundas Street West', 0.99)
(55, 'Grétrystraat 63', 0.99)
(62, '3 Chatham Street', 0.99)
(69, '319 N. Frances Street', 0.99)


In [29]:
query = '''
SELECT Title, Name
FROM albums
INNER JOIN artists 
ON artists.ArtistId = albums.ArtistId;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('For Those About To Rock We Salute You', 'AC/DC')
('Balls to the Wall', 'Accept')
('Restless and Wild', 'Accept')
('Let There Be Rock', 'AC/DC')
('Big Ones', 'Aerosmith')
('Jagged Little Pill', 'Alanis Morissette')
('Facelift', 'Alice In Chains')
('Warner 25 Anos', 'Antônio Carlos Jobim')
('Plays Metallica By Four Cellos', 'Apocalyptica')
('Audioslave', 'Audioslave')


In [30]:
query = '''
SELECT Name, Title
FROM artists
LEFT JOIN albums 
ON artists.ArtistId = albums.ArtistId
ORDER BY Name;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

('A Cor Do Som', None)
('AC/DC', 'For Those About To Rock We Salute You')
('AC/DC', 'Let There Be Rock')
('Aaron Copland & London Symphony Orchestra', 'A Copland Celebration, Vol. I')
('Aaron Goldberg', 'Worlds')
('Academy of St. Martin in the Fields & Sir Neville Marriner', 'The World of Classical Favourites')
('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner', 'Sir Neville Marriner: A Celebration')
('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair', 'Fauré: Requiem, Ravel: Pavane & Others')
('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart', 'Bach: Orchestral Suites Nos. 1 - 4')
('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett', None)


In [31]:
query = '''
SELECT tracks.trackid, tracks.name AS track, albums.title AS album, artists.name AS artist
FROM tracks
INNER JOIN albums 
ON albums.albumid = tracks.albumid
INNER JOIN artists 
ON artists.artistid = albums.artistid
WHERE artists.artistid = 10;
'''

cur.execute(query)
for c in cur.fetchmany(10):
    print(c)

(123, 'Quadrant', 'The Best Of Billy Cobham', 'Billy Cobham')
(124, "Snoopy's search-Red baron", 'The Best Of Billy Cobham', 'Billy Cobham')
(125, 'Spanish moss-"A sound portrait"-Spanish moss', 'The Best Of Billy Cobham', 'Billy Cobham')
(126, 'Moon germs', 'The Best Of Billy Cobham', 'Billy Cobham')
(127, 'Stratus', 'The Best Of Billy Cobham', 'Billy Cobham')
(128, 'The pleasant pheasant', 'The Best Of Billy Cobham', 'Billy Cobham')
(129, 'Solo-Panhandler', 'The Best Of Billy Cobham', 'Billy Cobham')
(130, 'Do what cha wanna', 'The Best Of Billy Cobham', 'Billy Cobham')


In [32]:
#https://www.sqlitetutorial.net/